In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import pandas as pd
import time
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score,recall_score, f1_score, roc_auc_score



In [14]:
df = pd.read_excel('employ_clean.xlsx')

In [ ]:

# Bắt đầu tính thời gian
start_time = time.time()

df['StartDate'] = pd.to_datetime(df['StartDate'], errors='coerce')
df['ExitDate'] = pd.to_datetime(df['ExitDate'], errors='coerce')

# Tính số năm làm việc (Work Duration)
df['WorkDuration'] = (df['ExitDate'].fillna(pd.to_datetime('today')) - df['StartDate']).dt.days / 365.25

# Mã hóa các cột phân loại
label_encoder = LabelEncoder()
df['IsEmployeeLeft'] = label_encoder.fit_transform(df['IsEmployeeLeft'])
df['GenderCode'] = label_encoder.fit_transform(df['GenderCode'])

# One-Hot Encoding cho cột 'Performance Score'
df = pd.get_dummies(df, columns=['Performance Score','JobFunctionDescription'], drop_first=True)

# Chọn đặc trưng và mục tiêu
X = df[['Age', 'GenderCode', 'WorkDuration', 'Current Employee Rating'] + [col for col in df.columns if 'Performance Score' in col or 'JobFunctionDescription' in col]]
y = df['IsEmployeeLeft']
start_time_rf = time.time()
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Xây dựng mô hình hồi quy logic
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Dự đoán và đánh giá mô hình
y_pred = model.predict(X_test)

training_time_rf = time.time() - start_time_rf

# Hiển thị kết quả

C:\Users\luan0\AppData\Local\Temp\ipykernel_5276\11375170.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['ExitDate'] = pd.to_datetime(df['ExitDate'], errors='coerce')


In [16]:
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)
# Giải thích các giá trị trong ma trận nhầm lẫn
TN, FP, FN, TP = conf_matrix.ravel()

[[349  73]
 [ 70 408]]


In [17]:
# Hiển thị kết quả
print(f'Training Time (Logistic Regression): {training_time_rf:.2f} seconds')
# Kết quả chi tiết về mô hình (precision, recall, f1-score, support)
classification_rep_dt = classification_report(y_test, y_pred)
print(classification_rep_dt)

Training Time (Logistic Regression): 0.13 seconds
              precision    recall  f1-score   support

           0       0.83      0.83      0.83       422
           1       0.85      0.85      0.85       478

    accuracy                           0.84       900
   macro avg       0.84      0.84      0.84       900
weighted avg       0.84      0.84      0.84       900



In [18]:
from sklearn.metrics import cohen_kappa_score

# Tính Cohen's Kappa
kappa_value = cohen_kappa_score(y_test, y_pred)

print(f'Cohen\'s Kappa: {kappa_value:.4f}')


Cohen's Kappa: 0.6809
